# Enrichissement du fichier excel

## Transfert du JSON vers l'excel

In [119]:
!pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [121]:
import json
from openpyxl import load_workbook
from datetime import datetime

# === 1. Charger le fichier JSON ===
with open("all_watches_data.json", "r", encoding="utf-8") as f:
    watch_data_list = json.load(f)

# === 2. Charger le fichier Excel existant ===
wb = load_workbook("Daytona-db-sample.xlsx")
sheet = wb.active

# === 3. Ajouter les données à partir de chaque montre ===
for entry in watch_data_list:
    if "data" not in entry:
        continue  # ou bien logguer: print(f"Entrée sans clé 'data': {entry}")
    
    data = entry["data"]
    metadata = data.get("metadata", {})
    details = data.get("watchDetail", {})

    next_row = sheet.max_row + 1


    # === 4. Construire les valeurs à insérer (dans le bon ordre) ===
    values = [
        entry.get("timestamp", ""),                                # Date
        details.get("yearOfProduction", ""),                       # Model year
        details.get("modelName", ""),                              # Model
        details.get("referenceNumber", ""),                        # Reference
        details.get("caseNumber", ""),                             # case #
        details.get("movementNumber", ""),                         # movement #
        metadata.get("jsonLdData", {}).get("description", ""),     # caliber (simplifié)
        details.get("caseMaterialName", ""),                       # Case material
        "",                                                        # Dial classification
        details.get("dialColorName", ""),                          # Dial color
        "", "", "", "",                                            # Dial 6, Dial text, SWISS, Dial detail
        details.get("bezelMaterialName", ""),                      # Bezel
        "", "",                                                    # Pushers, Hands
        "",                                                        # caseback inside
        "",                                                        # crown
        details.get("braceletMaterialName", ""),                   # Bracelet
        details.get("conditionName", ""),                          # Condition
        details.get("description", ""),                            # Comment
        details.get("createdDate", ""),                            # sold on
        details.get("countryName", ""),                            # Delivery country
        "", "", "",                                                # AD, 1st owner, last known location
        metadata.get("jsonLdData", {}).get("offers", {}).get("priceCurrency", ""),  # FS CURRENCY
        metadata.get("jsonLdData", {}).get("offers", {}).get("price", ""),          # FS price
        "", "",                                                    # Sold currency, Sold
        "everywatch.com",                                          # source
        metadata.get("image", "")                                  # Image
    ]

    # === 5. Écrire dans la feuille ===
    for col, val in enumerate(values, start=1):
        sheet.cell(row=next_row, column=col, value=val)

# === 6. Sauvegarder ===
wb.save("Daytona-db-sample.xlsx")


## Extraction d'infromations complémentaire via Open AI

In [122]:
pip install openai


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [123]:
!pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [140]:
import pandas as pd
import openai
import json
import re

### Lecture de l'excel via pandas afin de faciliter le traitement de données

In [125]:
df = pd.read_excel("Daytona-db-sample.xlsx")

In [126]:
df

,Date,Model year,Model,Reference,case #,movement #,caliber,Case material,Dial clssification,Dial color,...,Delivery country,AD,1st owner,last known location,FS CURRENCY,FS price,Sold currency,Sold,source,Image
0,2014/05,1988.0,Floating Cosmograph Mk 1,16520,R 944.844,NaN,4030,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,Christie's,HKD,80-120'000,HKD,150000,Christie's,no
1,2015/12,1989.0,Floating Cosmograph,16520,R 944.844,13390,4030,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,Phillips,NaN,NaN,HKD,106250,Phillips,yes
2,2019/05,1988.0,"Floating Cosmograph Mk 1 ""Tiffany & Co""",16520,R 944.914,NaN,4030,Steel,Mk 1,black / silver circles,...,USA,Tiffany & Co,NaN,Phillips,HKD,500-1'000'000,HKD,587500,Phillips,yes
3,2013/05,1989.0,Floating Cosmograph Mk 1,16520,R 944.941,NaN,4030,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,Sotheby's,CHF,10-12'000,CHF,12500,Sotheby's,no
4,2020/06,1988.0,Floating Cosmograph Mk 1,16520,R 975.963,12881,4030,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,Phillips,CHF,30-50'000,CHF,57500,Phillips,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,2025-06-12T20:13:07.597173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everywatch.com,NaN
4910,2025-06-12T20:13:12.785278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everywatch.com,NaN
4911,2025-06-12T20:13:17.807219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everywatch.com,NaN
4912,2025-06-12T20:13:23.033796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everywatch.com,NaN


### Définition d'une fonction "extract_info" pour lire et traiter les blocs de texte contenu dans les colonnes 'caliber' et 'Comment'

In [127]:
df['combined_text'] = df[['caliber', 'Comment']].astype(str).agg('\n'.join, axis=1)

In [128]:
df.columns

Index(['Date', 'Model year', 'Model', 'Reference', 'case #', 'movement #',
       'caliber', 'Case material', 'Dial clssification', 'Dial color',
       'Dial 6', 'Dial text', 'SWISS', 'Dial detail', 'Bezel', 'Pushers',
       'Hands', 'caseback inside', 'crown', 'Bracelet', 'Condition', 'Comment',
       'sold on', 'Delivery country', 'AD', '1st owner', 'last known location',
       'FS CURRENCY', 'FS price', 'Sold currency', 'Sold', 'source', 'Image',
       'combined_text'],
      dtype='object')

In [179]:
openai.api_key = "sk..." 

In [141]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [173]:
def extract_info(text):
    prompt = (
        "Tu es un assistant qui extrait les informations horlogères d’un texte "
        "et les retourne sous forme de dictionnaire JSON avec les clés suivantes : "
        "case_size, material, movement, year, glass, reference_number, serial_number, "
        "movement_number, case_number, limited_numbered, accessories, caliber, "
        "case_material, dial_classification, dial_color, dial_6, dial_text, swiss, "
        "dial_detail, bezel, pushers, hands, caseback_inside, crown, bracelet, condition, "
        "ad, first_owner, last_known_location. "
        "Si une information est absente, retourne null pour cette clé. "
        "Ne donne rien d’autre que le dictionnaire JSON."
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": text}
            ],
            temperature=0
        )
        content = response.choices[0].message.content

        # Fix : convertir les "null" non valides (ex : `name 'null' is not defined`)
        content = content.replace("null", "null")  # au cas où
        # Ajouter un bloc try alternatif si jamais c’est du pseudo-dict
        return json.loads(content)
    except Exception as e:
        return {"error": str(e)}

### Test de la fonction "extract_info" sur quelques lignes

#### Test sur 3 lignes (à partir de la ligne 121 car les lignes d'avant correspondent aux montres déjà scrappés par le client auparavent) 

In [174]:
from tqdm.notebook import tqdm
tqdm.pandas()

# Étape 1 – Appliquer sur 3 lignes seulement
parsed = df.loc[121:123, 'combined_text'].progress_apply(extract_info)

  0%|          | 0/3 [00:00<?, ?it/s]

In [175]:
# Étape 2 – Transformer en DataFrame
extracted_df = pd.json_normalize(parsed)
extracted_df.index = parsed.index  # pour aligner les index

In [176]:
# Étape 3 – Ajouter les colonnes dans le DataFrame d'origine
for col in extracted_df.columns:
    df.loc[extracted_df.index, col] = extracted_df[col]

In [177]:
extracted_df.index = parsed.index  # Important pour l’alignement

for col in extracted_df.columns:
    df.loc[extracted_df.index, col] = extracted_df[col]

In [178]:
df.loc[121:123, ['caliber', 'Comment'] + extracted_df.columns.tolist()]

,caliber,Comment,error
121,NaN,CIRCA: 2015\nCASE NO: 105'097\nCASE MATERIAL: ...,"You exceeded your current quota, please check ..."
122,NaN,CIRCA: 2012\nCASE NO: 3365\nCASE MATERIAL: Whi...,"You exceeded your current quota, please check ..."
123,NaN,CIRCA: 2010s\nCASE NO: 2733 T\nCASE MATERIAL: ...,"You exceeded your current quota, please check ..."


#### Test sur 10 lignes

In [144]:
parsed = df.loc[124:133, 'combined_text'].progress_apply(extract_info)

  0%|          | 0/10 [00:00<?, ?it/s]

In [137]:
# Étape 2 – Transformer en DataFrame
extracted_df = pd.json_normalize(parsed)
extracted_df.index = parsed.index  # pour aligner les index

# Étape 3 – Ajouter les colonnes dans le DataFrame d'origine
for col in extracted_df.columns:
    df.loc[extracted_df.index, col] = extracted_df[col]

# Vérification (facultative)
df.loc[124:133, ['caliber', 'Comment'] + extracted_df.columns.tolist()]

/var/folders/lj/8ml3hddd2hs_pgpwyfwl_y6r0000gn/T/ipykernel_29903/3353810636.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan None None None None None None 'Breguet strap' 'Gay Frères'
 'black crocodile with gold-plated JL pin buckle']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[extracted_df.index, col] = extracted_df[col]


,caliber,Comment,case_size,material,movement,year,glass,reference_number,serial_number,movement_number,...,accessories.papers,bracelet.material,bracelet.size,bracelet,accessories.clasp,dial_classification.type,dial_classification.details,accessories.with,accessories.buckle,accessories
124,None,CIRCA: 2000s\nCASE NO: 4297 N\nCASE MATERIAL: ...,25 x 37 mm,Yellow gold and diamond-set,Automatic,2000s,None,None,None,None,...,No,Yellow gold,Approx. 165 mm. overall length,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,None,CIRCA: 2000s\nCASE NO: 52'82I\nCASE MATERIAL: ...,35.5 mm,Yellow gold,Automatic,2000s,None,None,None,None,...,No,NaN,NaN,None,Yellow gold Breguet deployant clasp,NaN,NaN,NaN,NaN,NaN
126,None,CIRCA: 1990s\nCASE NO: 1466D\nCASE MATERIAL: Y...,35 mm,"Yellow gold, diamond and blue sapphire-set",Automatic,1990s,None,None,None,None,...,No,NaN,NaN,None,Yellow gold and diamond-set Breguet deployant ...,Mother of Pearl,"diamond-set indexes, guilloché center",NaN,NaN,NaN
127,None,CIRCA: 2004\nCASE NO: 1641 Q\nCASE MATERIAL: W...,40 mm,White gold,Automatic,2004,None,None,None,None,...,Yes,NaN,NaN,None,NaN,NaN,NaN,Breguet Certificate of Guarantee dated 28 July...,NaN,NaN
128,None,CIRCA: 2007\nCASE NO: 3885 Z\nCASE MATERIAL: W...,38 mm,White gold,Manual,2007,None,None,None,None,...,Yes,NaN,NaN,None,NaN,NaN,NaN,Breguet Certificate of Origin and Warranty dat...,White gold Breguet buckle,NaN
129,None,CIRCA: 2000s\nCASE NO: 2'664L\nCASE MATERIAL: ...,40 mm,None,Manual,2000s,None,None,None,None,...,No,NaN,NaN,None,Yellow gold Breguet deployant clasp,NaN,NaN,NaN,NaN,NaN
130,None,CIRCA: 2004\nCASE NO: 2914\nCASE MATERIAL: Yel...,36 mm,Yellow gold,Automatic,2004,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,Certificate of Origin and Warranty dated 28 Ju...
131,cal. 818/2,"Breguet, ref. 3910, small seconds watch in 750...",35 mm,750 gold,mechanical,None,None,3910,None,2198177,...,NaN,NaN,NaN,Breguet strap,NaN,NaN,NaN,NaN,NaN,black crocodile with 750 gold pin buckle
132,None,Chronograph wristwatch in 750 gold and steel c...,39 mm,750 gold and steel,automatic,None,None,3800,None,None,...,NaN,NaN,NaN,Gay Frères,NaN,NaN,NaN,NaN,NaN,None
133,839,Skeleton watch in gold metal containing a chis...,29 mm,gold metal,mechanical,None,None,None,None,2355284,...,NaN,NaN,NaN,black crocodile with gold-plated JL pin buckle,NaN,NaN,NaN,NaN,NaN,None


### Lancement de la fonction sur tout le dataframe (4914 lignes)

Étant donné le volume de la base de données le processus a été divisé par tranche de 500 lignes

In [157]:
def process_batch(df, start, end, max_workers=3):
    subset = df.iloc[start:end].copy()
    mask = subset['case_size'].isna() & subset['error'].isna()
    subset_to_process = subset.loc[mask]

    index_map = list(subset_to_process.index)
    results = [None] * len(subset_to_process)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(extract_info, text): i
            for i, text in enumerate(subset_to_process['combined_text'])
        }
        for future in tqdm(as_completed(futures), total=len(futures),
                           desc=f"GPT-4: lignes {start} à {end}"):
            idx = futures[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                results[idx] = {"error": str(e)}

    extracted_df = pd.json_normalize(results)
    extracted_df.index = subset_to_process.index

    for col in extracted_df.columns:
        subset.loc[extracted_df.index, col] = extracted_df[col]

    return subset

In [159]:
# Étape 0 — Ajout colonne 'error' si manquante
if 'error' not in df.columns:
    df['error'] = None

# Boucle traitement par blocs
for i in range(0, len(df), 500):
    end = min(i + 500, len(df))
    print(f"Traitement bloc {i} → {end}")
    updated_chunk = process_batch(df, i, end)
    updated_chunk.to_excel(f"chunk_{i}_{end}.xlsx", index=False)

Traitement bloc 0 → 500


GPT-4: lignes 0 à 500:   0%|          | 0/487 [00:00<?, ?it/s]

Traitement bloc 500 → 1000


GPT-4: lignes 500 à 1000:   0%|          | 0/500 [00:00<?, ?it/s]

Traitement bloc 1000 → 1500


...

GPT-4: lignes 3500 à 4000:   0%|          | 0/500 [00:00<?, ?it/s]

Traitement bloc 4000 → 4500


GPT-4: lignes 4000 à 4500:   0%|          | 0/500 [00:00<?, ?it/s]

Traitement bloc 4500 → 4914


GPT-4: lignes 4500 à 4914:   0%|          | 0/414 [00:00<?, ?it/s]

In [166]:
import os

# Lister tous les fichiers Excel batchés
batch_files = sorted([f for f in os.listdir() if f.startswith("chunk_") and f.endswith(".xlsx")])

# Charger chaque fichier et les concaténer
dfs = []
for file in batch_files:
    df_chunk = pd.read_excel(file)
    dfs.append(df_chunk)

# Fusionner tous les fichiers
df = pd.concat(dfs, ignore_index=True)

# Afficher un aperçu
df  # Tu peux mettre df.tail() ou df.sample(10) si tu préfères


,Date,Model year,Model,Reference,case #,movement #,caliber,Case material,Dial clssification,Dial color,...,bracelet.material,bracelet.size,dial_classification.type,dial_classification.details,accessories.buckle,accessories,error,bracelet.color,bracelet.clasp.type,bracelet.clasp.material
0,2014/05,1988.0,Floating Cosmograph Mk 1,16520,R 944.844,NaN,NaN,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/12,1989.0,Floating Cosmograph,16520,R 944.844,13390,NaN,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019/05,1988.0,"Floating Cosmograph Mk 1 ""Tiffany & Co""",16520,R 944.914,NaN,NaN,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013/05,1989.0,Floating Cosmograph Mk 1,16520,R 944.941,NaN,NaN,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020/06,1988.0,Floating Cosmograph Mk 1,16520,R 975.963,12881,NaN,Steel,Mk 1,black / silver circles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,2025-06-11T15:41:20.657520,NaN,Marine,5527TI/G2/5WV,NaN,NaN,Brand : Breguet\nStyle : Wristwatch\nDepartmen...,Titanium,NaN,Gray,...,NaN,NaN,NaN,NaN,NaN,NaN,"You exceeded your current quota, please check ...",NaN,NaN,NaN
4910,2025-06-11T15:41:25.494781,2023.0,Marine,5517TI/G2/5ZU,NaN,NaN,Brand : breguet\nDepartment : Men\nType : Wris...,Titanium,NaN,Gray,...,NaN,NaN,NaN,NaN,NaN,NaN,"You exceeded your current quota, please check ...",NaN,NaN,NaN
4911,2025-06-11T15:41:31.117191,2022.0,Marine,5527TI/Y1/5WV,NaN,NaN,Brand : Breguet\nDepartment : Men\nType : Wris...,Titanium,NaN,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,"You exceeded your current quota, please check ...",NaN,NaN,NaN
4912,2025-06-11T15:41:35.632176,NaN,Marine,5817ST/Y2/5V8,NaN,NaN,Brand : Breguet\nDepartment : men\nType : Wris...,SS,NaN,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,"You exceeded your current quota, please check ...",NaN,NaN,NaN
